# Introduction

- Notebook Author: [Trenton McKinney][1]
- Course: [DataCamp: Case Study: School Budgeting with Machine Learning in Python][2] - This notebook was created as a reproducible reference
- If you find the content beneficial, consider a [DataCamp Subscription][3]

[1]: https://trenton3983.github.io/
[2]: https://app.datacamp.com/learn/courses/case-study-school-budgeting-with-machine-learning-in-python
[3]: https://www.datacamp.com/pricing?period=yearly

# Course Description

Data science isn't just for predicting ad-clicks-it's also useful for social impact! This course is a case study from a machine learning competition on DrivenData. You'll explore a problem related to school district budgeting. By building a model to automatically classify items in a school's budget, it makes it easier and faster for schools to compare their spending with other schools. In this course, you'll begin by building a baseline model that is a simple, first-pass approach. In particular, you'll do some natural language processing to prepare the budgets for modeling. Next, you'll have the opportunity to try your own techniques and see how they compare to participants from the competition. Finally, you'll see how the winner was able to combine a number of expert techniques to build the most accurate model.

# Imports

- DataCamp package versions vs. local versions:
  - `sklearn v0.20.4` vs. `1.3.2`
  - `numpy v1.17.4` vs. `1.26.3`
  - `pandas v0.24.2` vs. `2.2.0`
  - `matplotlib v3.1.2` vs. `3.8.1`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Union
import os

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler, PolynomialFeatures
from sklearn.pipeline import FeatureUnion

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.metrics import make_scorer

In [ ]:
import sys
sys.path.append(r'D:\users\trenton\Dropbox\PythonProjects\DataCamp\functions')

from multilabel import multilabel_train_test_split
from score_sub import score_submission, _multi_multi_log_loss
from sparse_interactions import SparseInteractions

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

# Saving data from DataCamp to a local csv

```python
# output the data to a format that can be copied and pasted into notepad++
# find all non and replace with np.nan
# copy all of lists and replace ... in data = np.array([...])
# print the columns names df.columns.tolist() and then create a variable in a notebook
# create a dataframe with df = pd.DataFrame(data=data, columns=columns)

# create the dataframe in the DataCamp console
df = pd.read_csv('TrainingData.csv')  # do not specify index_col=0

# print each row as a list with a comma at the end and then copy all of the lists into notepad++
for r in df.to_numpy():
    print(list(r), ',')

# print the column names and copy them
df.columns.tolist()

# in jupyterlab
data = np.array([...])  # replace ... with the cleaned lists from notepad++
columns = ...  # pasted from DataCamp console

df = pd.DataFrame(data=data, columns=columns)

df.to_csv('TrainingData.csv', index=False)
```

# Exploring the Raw Data

In this chapter, you'll be introduced to the problem you'll be solving in this course. How do you accurately classify line-items in a school budget based on what that money is being used for? You will explore the raw text and numeric values in the dataset, both quantitatively and visually. And you'll learn how to measure success when trying to predict class labels for each row of the dataset.

In [ ]:
# set the current working directory if needed
cwd = os.getcwd()
if cwd != 'D:/Users/trenton/Dropbox/PythonProjects/DataCamp':
    os.chdir(r'D:\users\trenton\Dropbox\PythonProjects\DataCamp')

In [ ]:
df = pd.read_csv('data/2024-01-19_school_budgeting_with_machine_learning_in_python/TrainingData.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
df.info()

## Summarize the Data

In [ ]:
# Print the summary statistics
df.describe()

In [ ]:
# Create the histogram
plt.hist(df['FTE'].dropna(), ec='k')  # .dropna() isn't needed

# Add title and labels
plt.title('Distribution of %full-time \n employee works')
plt.xlabel('% of full-time')
plt.ylabel('num employees')

# Display the histogram
plt.show()

**The high variance in expenditures makes sense (some purchases are cheap some are expensive). Also, it looks like the FTE column is bimodal. That is, there are some part-time and some full-time employees.**

In [ ]:
df.dtypes.value_counts()

## Encode the Labels as Categorical Variables

In [ ]:
LABELS_td = ['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type', 'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status']
categorize_label = lambda x: x.astype('category')
df[LABELS_td] = df[LABELS_td].apply(categorize_label)
print(df[LABELS_td].dtypes)

## Counting Unique Labels

In [ ]:
num_unique_labels = df[LABELS_td].apply(pd.Series.nunique)
ax = num_unique_labels.plot(kind='bar', ylabel='Number of Unique Categories', xlabel='Column Name')

## `def compute_log_loss`

In [ ]:
def compute_log_loss(predicted: Union[float, List[float]], actual: [int, List[int]], eps: float=1e-14) -> float:
    """
    Computes the logarithmic loss between predicted and 
    actual when these are 1D arrays

    :param predicted: The predicted probabilities as floats between 0-1
    :param actual: The actual binary labels. Either 0 or 1
    :param eps (optional): log(0) is inf, so we need to offset our
                           precidted values slightly by eps from 0 to 1.
    """

    predicted = np.clip(predicted, eps, 1 - eps)
    loss = -1 * np.mean(actual * np.log(predicted)
                        + (1 - actual)
                        * np.log(1 - predicted))
    return loss

In [ ]:
data = [(0.85, 1), (0.99, 0), (0.51, 0)]

for p, y in data:
    print(compute_log_loss(p, y))

**Lowest: A, Middle: C, Highest: B**

**Of the two incorrect predictions, B will have a higher log loss because it is confident and wrong.**

## Computing log loss with Numpy

In [ ]:
correct_confident = np.array([0.95, 0.95, 0.95, 0.95, 0.95, 0.05, 0.05, 0.05, 0.05, 0.05])
correct_not_confident = np.array([0.65, 0.65, 0.65, 0.65, 0.65, 0.35, 0.35, 0.35, 0.35, 0.35])
wrong_not_confident = np.array([0.35, 0.35, 0.35, 0.35, 0.35, 0.65, 0.65, 0.65, 0.65, 0.65])
wrong_confident = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.95, 0.95, 0.95, 0.95, 0.95])
actual_labels = np.array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

In [ ]:
# Compute and print log loss for 1st case
correct_confident_loss = compute_log_loss(correct_confident, actual_labels)
print("Log loss, correct and confident: {}".format(correct_confident_loss)) 

# Compute log loss for 2nd case
correct_not_confident_loss = compute_log_loss(correct_not_confident, actual_labels)
print("Log loss, correct and not confident: {}".format(correct_not_confident_loss)) 

# Compute and print log loss for 3rd case
wrong_not_confident_loss = compute_log_loss(wrong_not_confident, actual_labels)
print("Log loss, wrong and not confident: {}".format(wrong_not_confident_loss)) 

# Compute and print log loss for 4th case
wrong_confident_loss = compute_log_loss(wrong_confident, actual_labels)
print("Log loss, wrong and confident: {}".format(wrong_confident_loss)) 

# Compute and print log loss for actual labels
actual_labels_loss = compute_log_loss(actual_labels, actual_labels)
print("Log loss, actual labels: {}".format(actual_labels_loss)) 

**Log loss penalizes highly confident wrong answers much more than any other type. This will be a good metric to use on your models.**

# Creating a simple first model

In this chapter, you'll build a first-pass model. You'll use numeric data only to train the model. Spoiler alert - throwing out all the text data is bad for performance! But you'll learn how to format your predictions. Then, you'll be introduced to natural language processing (NLP) in order to start working with the large amounts of text in the data.

## It's time to build a model

```python
data_to_train = df[NUMERIC_COLUMNS].fillna(-1000)
labels_to_use = pd.get_dummies(df[LABELS])

X_train, X_test, y_train, y_test = multilabel_train_test_split(data_to_train, labels_to_use, size=0.2, seed=123)

clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train, y_train)
```

### Setting up a train-test split in scikit-learn

In [ ]:
NUMERIC_COLUMNS_td = df.select_dtypes(include='number').columns.tolist()

# Create the new DataFrame: numeric_data_only
numeric_data_only = df[NUMERIC_COLUMNS_td].fillna(-1000)

# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df[LABELS_td])

# Create training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(numeric_data_only,
                                                               label_dummies,
                                                               size=0.2, 
                                                               seed=123)

# Print the info
print("X_train info:")
print(X_train.info())
print("\nX_test info:")  
print(X_test.info())
print("\ny_train info:")  
print(y_train.info())
print("\ny_test info:")  
print(y_test.info()) 

### Training a model

In [ ]:
# Instantiate the classifier: clf
clf = OneVsRestClassifier(LogisticRegression())

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Print the accuracy
print("Accuracy: {}".format(clf.score(X_test, y_test)))

**The good news is that your workflow didn't cause any errors. The bad news is that your model scored the lowest possible accuracy: 0.0! But hey, you just threw away ALL of the text data in the budget. Later, you won't. Before you add the text data, let's see how the model does when scored by log loss.**

## Making predictions

```python
houldout = pd.read_csv('HoldoutData.csv', index_col=0)
holdout = holdout[NUMERIC_COLUMNS].fillna(-1000)
predictions = clf.predict_proba(holdout)
```

- Using `.predict()`
  - would result in an output of 0 or 1
  - Log loss penalized being confident and wrong
  - Worse performance compared to `.predict_proba()`

- Format and submit predictions
```python
prediction_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS], prefix_sep='__').columns, index=houldout.index, data=predictions)
prediction_df.to_csv('predictions.csv')
score = score_submission(pred_path='predictions.csv')
```

### Use your model to predict values on holdout data

In [ ]:
# Instantiate the classifier: clf
clf = OneVsRestClassifier(LogisticRegression())

# Fit it to the training data
clf.fit(X_train, y_train)

# Load the holdout data: holdout
holdout = pd.read_csv('data/2024-01-19_school_budgeting_with_machine_learning_in_python/HoldoutData.csv', index_col=0)

# Generate predictions: predictions
NUMERIC_COLUMNS_hd = holdout.select_dtypes(include='number').columns
predictions = clf.predict_proba(holdout[NUMERIC_COLUMNS_hd].fillna(-1000))

### Writing out your results to a csv for submission

In [ ]:
# Format predictions in DataFrame: prediction_df
# pd.get_dummies(df[LABELS], ...) is correctly using df, not holdout
prediction_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS_td], prefix_sep='__').columns,
                             index=holdout.index,
                             data=predictions)

# Save prediction_df to csv
prediction_df.to_csv('data/2024-01-19_school_budgeting_with_machine_learning_in_python/predictions.csv')

In [ ]:
# requires functions and variables from https://goodboychan.github.io/python/datacamp/machine_learning/2020/06/05/01-School-Budgeting-with-Machine-Learning-in-Python.html
# BOX_PLOTS_COLUMN_INDICES, def _multi_multi_log_loss, and def score_submission

# Submit the predictions for scoring: score
score = score_submission(pred_path='data/2024-01-19_school_budgeting_with_machine_learning_in_python/predictions.csv', holdout_path='data/2024-01-19_school_budgeting_with_machine_learning_in_python/TestSetLabelsSample.csv')

# Print score
print('Your model, trained with numeric data only, yields logloss score: {}'.format(score))

**Even though your basic model scored 0.0 accuracy, it nevertheless performs better than the benchmark score of 2.0455. You've now got the basics down and have made a first pass at this complicated supervised learning problem. It's time to step up your game and incorporate the text data.**

## A very brief introduction to NLP

- Data for NLP:
  - Text, documents, speech, ...
- Tokenization
  - Splitting a string into segments
  - Store segments as list
- Example: "Natural Language Processing"
  - `['Natural', 'Language', 'Processing']`

- Tokenize on whitespace
  - Petro-vend fuel and fluids
    - Petro-vend | fuel | and | fluids
- Tokenize on whitespace and punctuation
  - Petro | vend | fuel | and | fluids
 
- Bag of words representation
  - Count the umber of times a particular token appears
  - 'Bag of words'
    - Count the number of times a word was pulled out of the bag
- The approach discards information about word order
  - 'Red, not blue' is the same as 'blue, not red'
 
- 1-gram, 2-gram, ..., n-gram
  - ![][1]
 
[1]: https://raw.githubusercontent.com/trenton3983/DataCamp/master/Images/2024-01-19_school_budgeting_with_machine_learning_in_python/sb01.jpg

### Tokenizating text

- 6

### Testing your NLP credentials with n-grams

- 12 - The number of `1-grams + 2-grams + 3-grams` is `5 + 4 + 3 = 12`.

## Representing text numerically

**Scikit-learn tools for bag-of-words**

- `CountVectorizer()`
  - Tokenizes all the strings
  - Builds a 'vocabulary'
  - Counts the occurrences of each token in the vocabulary
 
**Using `CountVectorizer() on a column of main dataset**

```python
from sklearn.feature_extraction.text import CountVecotrizer

TOKENS_BASIC = '\\\\S+(?=\\\\s+)'
df.Program_Description.fillna('', inplace=True)
vec_basic = CountVectorizer(token_pattern=TOKENS_BASIC)

vec_basic.fit(df.Program_Description)
msg = 'There are {} token in Program_Desction if tokean are any non-whitespace'
print(msg.format(len(vec_basic.get_feature_names())))
```


### Creating a bag-of-words in scikit-learn

In this exercise, you'll study the effects of tokenizing in different ways by comparing the bag-of-words representations resulting from different token patterns.

You will focus on one feature only, the `Position_Extra` column, which describes any additional information not captured by the `Position_Type` label.

For example, in the Shell you can check out the budget item in row 8960 of the data using `df.loc[8960]`. Looking at the output reveals that this `Object_Description` is overtime pay. For who? The Position Type is merely "other", but the Position Extra elaborates: "BUS DRIVER". Explore the column further to see more instances. It has a lot of NaN values.

Your task is to turn the raw text in this column into a bag-of-words representation by creating tokens that contain only alphanumeric characters.

For comparison purposes, the first 15 tokens of `vec_basic`, which splits `df.Position_Extra` into tokens when it encounters only whitespace characters, have been printed along with the length of the representation.

**Instructions**

- Import `CountVectorizer` from `sklearn.feature_extraction.text`.
- Fill missing values in `df.Position_Extra` using `.fillna('')` to replace NaNs with empty strings. Specify the additional keyword argument `inplace=True` so that you don't have to assign the result back to `df`.
- Instantiate the `CountVectorizer` as `vec_alphanumeric` by specifying the `token_pattern` to be `TOKENS_ALPHANUMERIC`.
- Fit `vec_alphanumeric` to `df.Position_Extra`.
- Hit submit to see the `len` of the fitted representation as well as the first 15 elements, and compare to `vec_basic`.

In [ ]:
# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Fill missing values in df.Position_Extra
df.fillna({'Position_Extra': ''}, inplace=True)

# Instantiate the CountVectorizer: vec_alphanumeric
vec_alphanumeric = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Fit to the data
vec_alphanumeric.fit(df.Position_Extra)

# Print the number of tokens and first 15 tokens
msg = "There are {} tokens in Position_Extra if we split on non-alpha numeric"
print(msg.format(len(vec_alphanumeric.get_feature_names_out())))
print(vec_alphanumeric.get_feature_names_out()[:15])

**Treating only alpha-numeric characters as tokens gives you a smaller number of more meaningful tokens.**

### Combining text columns for tokenization

In order to get a bag-of-words representation for all the text data in our DataFrame, you must first convert the text data in each row of the DataFrame into a single string.

In the previous exercise, this wasn't necessary because you only looked at one column of data, so each row was already just a single string. `CountVectorizer` expects each row to just be a single string, so in order to use all of the text columns, you'll need a method to turn a list of strings into a single string.

In this exercise, you'll complete the function definition `combine_text_columns()`. When completed, this function will convert all training text data in your DataFrame to a single string per row that can be passed to the vectorizer object and made into a bag-of-words using the `.fit_transform()` method.

Note that the function uses `NUMERIC_COLUMNS` and `LABELS` to determine which columns to drop. These lists have been loaded into the workspace.

**Instructions**

- Use the `.drop()` method on `data_frame` with `to_drop` and `axis=` as arguments to drop the non-text data. Save the result as `text_data`.
- Fill in missing values (inplace) in `text_data` with blanks (""), using the `.fillna()` method.
- Complete the `.apply()` method by writing a lambda function that uses the `.join()` method to join all the items in a row with a space in between.

In [ ]:
# Define combine_text_columns()
def combine_text_columns(data_frame: pd.DataFrame, to_drop: list=NUMERIC_COLUMNS_td + LABELS_td) -> pd.Series:
    """ converts all text in each row of data_frame to single vector """
    
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    
    # Replace nans with blanks
    text_data.fillna('', inplace=True)
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

### What's in a token?

Now you will use `combine_text_columns` to convert all training text data in your DataFrame to a single vector that can be passed to the vectorizer object and made into a bag-of-words using the `.fit_transform()` method.

You'll compare the effect of tokenizing using any non-whitespace characters as a token and using only alphanumeric characters as a token.

**Instructions**

- Import `CountVectorizer` from `sklearn.feature_extraction.text`.
- Instantiate `vec_basic` and `vec_alphanumeric` using, respectively, the `TOKENS_BASIC` and `TOKENS_ALPHANUMERIC` patterns.
- Create the text vector by using the `combine_text_columns()` function on `df`.
- Using the `.fit_transform()` method with `text_vector`, fit and transform first `vec_basic` and then `vec_alphanumeric`. Print the number of tokens they contain.

In [ ]:
# Create the basic token pattern
TOKENS_BASIC = '\\S+(?=\\s+)'

# Create the alphanumeric token pattern
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate basic CountVectorizer: vec_basic
vec_basic = CountVectorizer(token_pattern=TOKENS_BASIC)

# Instantiate alphanumeric CountVectorizer: vec_alphanumeric
vec_alphanumeric = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Create the text vector
text_vector = combine_text_columns(df)

# Fit and transform vec_basic
vec_basic.fit_transform(text_vector)

# Print number of tokens of vec_basic
print("There are {} tokens in the dataset".format(len(vec_basic.get_feature_names_out())))

# Fit and transform vec_alphanumeric
vec_alphanumeric.fit_transform(text_vector)

# Print number of tokens of vec_alphanumeric
print("There are {} alpha-numeric tokens in the dataset".format(len(vec_alphanumeric.get_feature_names_out())))

**Notice that tokenizing on alpha-numeric tokens reduced the number of tokens, just as in the last exercise. We'll keep this in mind when building a better model with the Pipeline object next.**

# Improving your model

Here, you'll improve on your benchmark model using pipelines. Because the budget consists of both text and numeric data, you'll learn how to build pipelines that process multiple types of data. You'll also explore how the flexibility of the pipeline workflow makes testing different approaches efficient, even in complicated problems like this one!

## Pipelines, feature & text preprocessing

1. **Pipelines, feature & text preprocessing**: You've submitted your first simple model. Now it's time to combine what we've learned about NLP with our model pipeline and incorporate the text data into our algorithm.
- Repeatable way to go from raw data to trained model
- Pipeline object takes sequential list of steps
  - Output of one step is input to next step
- Each step is a tuple with two elements
  - Name: string
  - Transform: obj implementing .fit() and .transform()
- Flexible: a step can itself be another pipeline!

2. **The pipeline workflow**: The supervised learning course introduced pipelines, which are a repeatable way to go from raw data to a trained machine learning model. The scikit-learn Pipeline object takes a sequential list of steps where the output of one step is the input to the next. Each step is represented with a name for the step, that is simply a string, and an object that implements the fit and the transform methods. A good example is the `CountVectorizer` that we used earlier. As we'll see Pipelines are a very flexible way to represent your workflow. In fact, you can even have a sub-pipeline as one of the steps! The beauty of the Pipeline is that it encapsulates every transformation from raw data to a trained model.

```python
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
pl = Pipeline([('clf', OneVsRestClassifier(LogisticRegression()))])
```

3. **Instantiate simple pipeline with one step**: After importing the relevant modules, we'll start with a one-step pipeline. Although we don't need a pipeline for a single step, this exercise will get us familiar with the syntax. Remember, we create a Pipeline by passing it a series of named steps. In this case the name is the string 'clf', and the step is the one-vs-rest logistic regression classifier we created earlier.

4. **Train and test with sample numeric data**: The sample dataset we've been working with has numeric data in the numeric columns and the with missing column. We'll start by building a model to predict the label column with the numeric data.

5. **Train and test with sample numeric data**: First, we'll use the train_test split function from sklearn dot model_selection. Our X will just be the numeric column, and our Y will be the dummy encoding of the label column. We'll call the fit method on our Pipeline, just like a normal classifier in sklearn. We can see that it returns a Pipeline that has one step to do our logistic regression.

```python
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric']], pd.get_dummies(sample_df['label']), random_state=2)
pl.fit(X_train, y_train)
```

6. **Train and test with sample numeric data**: By using the score function on our pipeline, we can see how our Pipeline performs on the test set. The default scoring method for this classifier is accuracy. That's good enough for our sample dataset.

```python
accuracy = pl.score(X_test, y_test)
print('accuracy on numeric data, no nans: ', accuracy)
```

7. **Adding more steps to the pipeline**: Let's add the with_missing column to our training set. Oops! Now when we call fit, we see a value error that explains that our input has NaN values.

```python
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing']], pd.get_dummies(sample_df['label']), random_state=2)
pl.fit(X_train, y_train)
```

8. **Preprocessing numeric features with missing data**: To address this, we'll add an Imputer to our pipeline, which will fill in the NaN values. To do so, we add a step to the Pipeline with the name 'imp' and an Imputer object. The default imputation in scikit-learn is to fill missing values with the mean of the column in question. You can look at the documentation of the Imputer object to see other possible imputation strategies.

```python
from sklearn.preprocessing import Imputer
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing']], pd.get_dummies(sample_df['label']), random_state=2)
pl = Pipeline([('imp', Imputer()), ('clf', OneVsRestClassifier(LogisticRegression()))])
```

9. **Preprocessing numeric features with missing data**: We can call fit and score on the new pipeline, just like we did before. But, in this case our model also includes the column with_missing which had NaN values that were imputed.

```python
pipeline.fit(X_train, y_train)
accuracy = pl.score(X_test, y_test)
print('accuracy on all numeric, incl nans: ', accuracy)
```

### Instantiate pipeline

To make your life easier as you start to work with all the data in your original DataFrame, `df`, it's time to turn to one of scikit-learn's most useful objects: the `Pipeline`.

For the next few exercises, you'll reacquaint yourself with pipelines and train a classifier on some synthetic (sample) data of multiple datatypes before using the same techniques on the main dataset.

The sample data is stored in the DataFrame, `sample_df`, which has three kinds of feature data: numeric, text, and numeric with missing values. It also has a label column with two classes, `a` and `b`.

In this exercise, your job is to instantiate a pipeline that trains using the `numeric` column of the sample data.

**Instructions**

- Import `Pipeline` from `sklearn.pipeline`.
- Create training and test sets using the numeric data only. Do this by specifying `sample_df[['numeric']]` in `train_test_split()`.
- Instantiate a pipeline as pl by adding the classifier step. Use a name of `'clf'` and the same classifier from Chapter 2: `OneVsRestClassifier(LogisticRegression())`.
- Fit your pipeline to the training data and compute its accuracy to see it in action! Since this is toy data, you'll use the default scoring method for now. In the next chapter, you'll return to log loss scoring.

In [ ]:
sample_df = pd.read_csv('data/2024-01-19_school_budgeting_with_machine_learning_in_python/sample_df.csv')

sample_df.fillna({'text': ''}, inplace=True)

In [ ]:
# Split and select numeric data only, no nans 
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric']], pd.get_dummies(sample_df['label']),  random_state=22)

# Instantiate Pipeline object: pl
pl = Pipeline([('clf', OneVsRestClassifier(LogisticRegression()))])

# Fit the pipeline to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("Accuracy on sample data - numeric, no nans: ", accuracy)

**Now it's time to incorporate numeric data with missing values by adding a preprocessing step!**

### Preprocessing numeric features

What would have happened if you had included the with `'with_missing'` column in the last exercise? Without imputing missing values, the pipeline would **not** be happy (try it and see). So, in this exercise you'll improve your pipeline a bit by using the `Imputer()` imputation transformer from scikit-learn to fill in missing values in your sample data.

By default, the [imputer transformer][1] replaces NaNs with the mean value of the column. That's a good enough imputation strategy for the sample data, so you won't need to pass anything extra to the imputer.

After importing the transformer, you will edit the steps list used in the previous exercise by inserting a `(name, transform)` tuple. Recall that steps are processed sequentially, so make sure the new tuple encoding your preprocessing step is put in the right place.

The `sample_df` is in the workspace, in case you'd like to take another look. Make sure to select **both** numeric columns- in the previous exercise we couldn't use `with_missing` because we had no preprocessing step!

**Instructions**

- Import `Imputer` from `sklearn.preprocessing`.
- Create training and test sets by selecting the correct subset of `sample_df`: `numeric` and `with_missing`.
- Add the tuple `('imp', Imputer())` to the correct position in the pipeline. `Pipeline` processes steps sequentially, so the imputation step should come __before__ the classifier step.
- Complete the `.fit()` and `.score()` methods to fit the pipeline to the data and compute the accuracy.

[1]: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html

In [ ]:
# Create training and test sets using only numeric data
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing']],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=456)

# Insantiate Pipeline object: pl
pl = Pipeline([('imp', SimpleImputer()), ('clf', OneVsRestClassifier(LogisticRegression()))])

# Fit the pipeline to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("Accuracy on sample data - all numeric, incl nans: ", accuracy)

**Now you know how to use preprocessing in pipelines with numeric data, and it looks like the accuracy has improved because of it! Text data preprocessing is next!**

## Test features and feature union

1. **Text features and feature unions**: In this video, we'll look at how we process text data in a pipeline and then how we put it all together.

2. **Preprocessing text features**: Our sample dataframe contains one column we haven't used yet, the text column. First, we'll change our train and test data just to be working with the text data for now. Then, we'll add a step to our Pipeline with the name vec and the CountVectorizer object from scikit-learn. Now our pipeline will use the CountVectorizer on our dataset, and then it will pass the result into our classifier.

```python
from sklearn.feature_extraction.text import CountVectorizer
X_train, X_test, y_train, y_test = train_test_split(sample_df['text'], pd.get_dummies(sample_df['label']), random_state=2)
pl = Pipeline([('vec', CountVectorizer()), ('clf', OneVsRestClassifier(LogisticRegression()))])
```

3. **Preprocessing text features**: Again, we can call the fit and score methods on the Pipeline. The score function now reports our accuracy if we just use the text data.

```python
pl.fit(X_train, y_train)

accuracy = pl.score(X_test, y_test)
print('accuracy on sample data: ', accuracy)
```

4. **Preprocessing multiple dtypes**: Let's say we want to use all of our data in a single pipeline. We can't just have a Pipeline that has a `CountVectorizer` step, Imputation step, and then a classifier. The `CountVectorizer` won't know what to do with the numeric columns, and we don't want to perform imputation on the text columns. In order to build our pipeline, we need to separately operate on the text columns and on the numeric columns. There are two tools: FunctionTransformer and FeatureUnion that will help us build a Pipeline to work with both our text and numeric data. The first utility that we cover is the FunctionTransformer.
- Want to use all available features in one pipeline
- Problem
  - Pipeline steps for numeric and text preprocessing can't follow each other
  - e.g., output of `CountVectorizer` can't be input to `Imputer`
- Solution
  - `FunctionTransformer()` & `FeatureUnion()`

5. **FunctionTransformer**: `FunctionTransformer` has a simple job: take a Python function, and turn it into an object that the scikit-learn Pipeline can understand. We'll write two simple functions: one that takes the whole dataframe, and returns just the numeric columns. The other will take the whole dataframe and return just the text columns. Using these function transformers, we can build a separate Pipeline for our numeric data and for our text data.
- Turns a Python function into an object that a scikit-learn pipeline can understand
- Need to write two functions for pipeline preprocessing
  - Take entire DataFrame, return numeric columns
  - Take entire DataFrame, return text columns
- Can then preprocess numeric and text data in separate pipelines

6. **Putting it all together**: First, we'll do our train_test split on the entire dataset and import the FunctionTransformer and FeatureUnion utilities.

```python
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric','with_missing', 'text']], pd.get_dummies(sample_df['label']), random_state=2)

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
```

7. **Putting it all together**: Next, we'll create two `FunctionTransformer` objects. The first takes a dataframe and just returns the column named 'text'. The second takes a dataframe and returns the columns named 'numeric' and 'with_missing'. These two function transformer objects will let us set up separate Pipelines that operate on the selected columns only. Note that we've passed the parameter validate equals False. This simply tells scikit-learn it doesn't need to check for NaNs or validate the dtypes of the input. We'll do that work ourselves.

```python
get_text_data = FunctionTransformer(lambda x: x['text'], validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[['numeric', 'with_missing']], validate=False)
```

8. **FeatureUnion Text and Numeric Features**: `FeatureUnion` from the sklearn dot pipeline module is the other utility we need. We know that our text pipeline generates the array on the left, our text features.

9. **FeatureUnion Text and Numeric Features**: And our numeric pipeline generates the array on the right, our numeric features.

10. **FeatureUnion Text and Numeric Features**: The FeatureUnion object puts these two sets of features together as a single array that will be the input to our classifier.

```python
from sklearn.pipeline import FeatureUnion
union = FeatureUnion([('numeric', numeric_pipeline), ('text', text_pipeline)])
```

11. **Putting it all together**: Here's our entire pipeline, which processes our text and numeric data. First, we create our separate text and numeric pipelines. Remember, `get_numeric_data` and `get_text_data` are the `FunctionTransformers` that we just created. These pipelines output our numeric features and our text features respectively. Then we create our overall pipeline, which has two steps: First, the `FeatureUnion` takes a list of objects, calls each one, and then concatenates the output into a wide array out of all the results. In this case, it will call our numeric pipeline and then our text pipeline. Once we have this array of all of our features, we can pass it to our classifier. We can call fit and score on this pipeline, just like our simpler ones before it.

```python
numeric_pipeline = Pipeline([('selector', get_numeric_data), ('imputer', Imputer())])
text_pipeline = Pipeline([('selector', get_text_data), ('vectorizer', CountVectorizer())])
pl = Pipeline([('union', FeatureUnion([('numeric', numeric_pipeline), ('text', text_pipeline)])),
               ('clf', OneVsRestClassifier(LogisticRegression()))])
```

### Preprocessing text features

Here, you'll perform a similar preprocessing pipeline step, only this time you'll use the `text` column from the sample data.

To preprocess the text, you'll turn to `CountVectorizer()` to generate a bag-of-words representation of the data, as in Chapter 2. Using the [default][1] arguments, add a `(step, transform)` tuple to the steps list in your pipeline.

Make sure you select only the text column for splitting your training and test sets.

As usual, your `sample_df` is ready and waiting in the workspace.

**Instructions**

- Import `CountVectorizer` from `sklearn.feature_extraction.tex`t.
- Create training and test sets by selecting the correct subset of `sample_df`: `'text'`.
- Add the `CountVectorizer` step (with the name `'vec'`) to the correct position in the pipeline.
- Fit the pipeline to the training data and compute its accuracy.

[1]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
# Split out only the text data
X_train, X_test, y_train, y_test = train_test_split(sample_df['text'],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=456)

# Instantiate Pipeline object: pl
pl = Pipeline([
        ('vec', CountVectorizer()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - just text data: ", accuracy)

### Multiple types of processing: FunctionTransformer

The next two exercises will introduce new topics you'll need to make your pipeline truly excel.

Any step in the pipeline must be an object that implements the `fit` and `transform` methods. The [`FunctionTransformer`][1] creates an object with these methods out of any Python function that you pass to it. We'll use it to help select subsets of data in a way that plays nicely with pipelines.

You are working with numeric data that needs imputation, and text data that needs to be converted into a bag-of-words. You'll create functions that separate the text from the numeric variables and see how the `.fit()` and `.transform()` methods work.

**Instructions**

- Compute the selector `get_text_data` by using a lambda function and `FunctionTransformer()` to obtain all `'text'` columns.
- Compute the selector `get_numeric_data` by using a lambda function and `FunctionTransformer()` to obtain all the numeric columns (including missing data). These are `'numeric'` and `'with_missing'`.
- Fit and transform `get_text_data` using the `.fit_transform()` method with `sample_df` as the argument.
- Fit and transform `get_numeric_data` using the same approach as above.


[1]: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html

In [ ]:
# Obtain the text data: get_text_data
get_text_data = FunctionTransformer(lambda x: x['text'], validate=False)

# Obtain the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[['numeric', 'with_missing']], validate=False)

# Fit and transform the text data: just_text_data
just_text_data = get_text_data.fit_transform(sample_df)

# Fit and transform the numeric data: just_numeric_data
just_numeric_data = get_numeric_data.fit_transform(sample_df)

# Print head to check results
print('Text Data')
display(just_text_data.head())
print('\nNumeric Data')
display(just_numeric_data.head())

### Multiple types of processing: FeatureUnion

Now that you can separate text and numeric data in your pipeline, you're ready to perform separate steps on each by nesting pipelines and using `FeatureUnion()`.

These tools will allow you to streamline all preprocessing steps for your model, even when multiple datatypes are involved. Here, for example, you don't want to impute our text data, and you don't want to create a bag-of-words with our numeric data. Instead, you want to deal with these separately and then join the results together using `FeatureUnion()`.

In the end, you'll still have only two high-level steps in your pipeline: preprocessing and model instantiation. The difference is that the first preprocessing step actually consists of a pipeline for numeric data and a pipeline for text data. The results of those pipelines are joined using `FeatureUnion()`.

**Instructions**

- In the `process_and_join_features`:
  - Add the steps `('selector', get_numeric_data)` and `('imputer', Imputer())` to the `'numeric_features'` preprocessing step.
  - Add the equivalent steps for the `text_features` preprocessing step. That is, use `get_text_data` and a `CountVectorizer` step with the name `'vectorizer'`.
- Add the transform step `process_and_join_features` to `'union'` in the main pipeline, `pl`.
- Hit submit to see the pipeline in action!

In [ ]:
# Split using ALL data in sample_df
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing', 'text']],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=22)

# Create a FeatureUnion with nested pipeline: process_and_join_features
process_and_join_features = FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )

# Instantiate nested pipeline: pl
pl = Pipeline([
        ('union', process_and_join_features),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])


# Fit pl to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - all data: ", accuracy)

## Choosing a classification model

1. **Choosing a classification model**: Now that we understand how to pull together text and numeric data into a single machine learning pipeline, we'll return to talking about our school budget dataset. In the sample dataset, there was only one text column. This is the format that `CountVectorizer` expects.

2. **Main dataset: lots of text**: However, our main dataset has 14 text columns. In an interactive exercise during our NLP chapter we wrote a function combine_text_columns that put together all the text columns into a single column. We'll re-use this function as part of our Pipeline to process the text in the budget dataset.

```python
LABELS = ['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type', 'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status']
NON_LABELS = [c for c in df.columns if c not in LABELS]
len(NON_LABELS) - len(NUMERIC_COLUMNS)
```

3. **Using pipeline with the main dataset**: Again, as we did in the last chapter, we'll use the `get_dummies` function from pandas to create our label array, and we'll also create our train-test split using the `multilabel_train_test_split` function. Hopefully, this is starting to feel familiar.

4. **Using pipeline with the main dataset**: Now we'll create a pipeline for working with our main dataset. The beauty of this code, is that it has one change from the code we used for the sample dataset: we create a `FunctionTransformer` object using our combine_text_columns function instead of the simple selection function we used in the sample dataset. Other than this one change, the pipeline that processes the text, imputes the numeric columns, joins those results together and then fits a multilabel logistic regression remains the same.

```python
import numpy as np
import pandas as pd

df = pd.read_csv('TrainingSetSample.csv', index_col=0)
dummy_labels = pd.get_dummies(df[LABELS])
X_train, X_test, y_train, y_test = multilabel_train_test_split(
    df[NON_LABELS], dummy_labels,
    0.2)

get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate=False)
pl = Pipeline([
    ('union', FeatureUnion([
        ('numeric_features', Pipeline([
            ('selector', get_numeric_data),
            ('imputer', Imputer())
        ])),
        ('text_features', Pipeline([
            ('selector', get_text_data),
            ('vectorizer', CountVectorizer())
        ]))
    ])
     ),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])
```

5. **Performance using main dataset**: We can now fit this Pipeline on the school budget dataset. As part of our exercises, we'll look at the results of this pipeline. Now we have infrastructure for processing our features and fitting a model. This infrastructure allows us to easily experiment with adapting different parts of the pipeline. Part of your challenge in the exercises will be to improve the performance of the pipeline.

```python
pl.fit(X_train, y_train)
```

6. **Flexibility of model step**: For example, we can easily experiment with different classes of models. All of our Pipeline code can stay the same, but we can update the last step to be a different class of model instead of `LogisticRegression`. For example, we could try a `RandomForestClassifier`, `NaiveBayesClassifier`, or a `KNeighborsClassifier` instead. In fact, you can look at the scikit-learn documentation and choose any model class you want!
- Is current model the best?
- Can quickly try different models with pipelines
  - Pipeline preprocessing steps unchanged
  - Edit the model step in your pipeline
  - Random Forest, Naive Bayes, k-NN

7. **Easily try new models using pipeline**: Here's an example of how simple it is to change the classification method in our scikit-learn Pipeline. We simply import a different class of model--in this case, we'll use a `RandomForestClassifier`. Then, the only other line that needs to change is the one that defines the classifier inside the Pipeline. This makes it very simple for us to try a large number of different model classes and determine which one is the best for the problem that we're working on.

```python
from sklearn.ensemble import RandomForestClassifier

pl = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('imputer', Imputer())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vectorizer', CountVectorizer())
            ]))
        ]
    )),
    ('clf', OneVsRest(RandomForestClassifier()))
])
```

8. **Let's practice**: Now your task is to implement the classification pipeline on the school budget dataset. You will also get the chance to experiment with the Pipeline steps and model class. See how much you can improve the performance of the model!

### Using FunctionTransformer on the main dataset

In this exercise you're going to use `FunctionTransformer` on the primary budget data, before instantiating a multiple-datatype pipeline in the next exercise.

Recall from Chapter 2 that you used a custom function `combine_text_columns` to select and properly format **text data** for tokenization; it is loaded into the workspace and ready to be put to work in a function transformer!

Concerning the **numeric data**, you can use `NUMERIC_COLUMNS`, preloaded as usual, to help design a subset-selecting lambda function.

You're all finished with sample data. The original `df` is back in the workspace, ready to use.

**Instructions**

- Complete the call to `multilabel_train_test_split()` by selecting `df[NON_LABELS]`.
- Compute `get_text_data` by using `FunctionTransformer()` and passing in `combine_text_columns`. Be sure to also specify `validate=False`.
- Use `FunctionTransformer()` to compute `get_numeric_data`. In the lambda function, select out the `NUMERIC_COLUMNS` of `x`. Like you did when computing `get_text_data`, also specify `validate=False`.

In [ ]:
df = pd.read_csv('data/2024-01-19_school_budgeting_with_machine_learning_in_python/TrainingData.csv', index_col=0)
LABELS_td = ['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type', 'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status']
categorize_label = lambda x: x.astype('category')
df[LABELS_td] = df[LABELS_td].apply(categorize_label)
NUMERIC_COLUMNS_td = df.select_dtypes(include='number').columns.tolist()

In [ ]:
# Get the dummy encoding of the labels
dummy_labels = pd.get_dummies(df[LABELS_td])

# Get the columns that are features in the original df
NON_LABELS = [c for c in df.columns if c not in LABELS_td]

# Split into training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               dummy_labels,
                                                               0.2, 
                                                               seed=123)

# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(combine_text_columns, validate=False)

# Preprocess the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS_td], validate=False)

### Add a model to the pipeline

You're about to take everything you've learned so far and implement it in a `Pipeline` that works with the real, [DrivenData][1] budget line item data you've been exploring.

**Surprise!** The structure of the pipeline is exactly the same as earlier in this chapter:

- the **preprocessing step** uses `FeatureUnion` to join the results of nested pipelines that each rely on `FunctionTransformer` to select multiple datatypes
- the **model step** stores the model object

- You can then call familiar methods like `.fit()` and `.score()` on the `Pipeline` object `pl`.

**Instructions**

- Complete the `'numeric_features'` transform with the following steps:
  - `get_numeric_data`, with the name `'selector'`.
  - `Imputer()`, with the name `'imputer'`.
- Complete the `'text_features'` transform with the following steps:
  - `get_text_data`, with the name `'selector'`.
  - `CountVectorizer()`, with the name `'vectorizer'`.
- Fit the pipeline to the training data.
- Hit submit to compute the accuracy!


[1]: https://www.drivendata.org/

In [ ]:
# Complete the pipeline: pl
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000)))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on budget dataset: ", accuracy)

**Now that you've built the entire pipeline, you can easily start trying out different models by just modifying the `'clf'` step.**

### Try a different class of model

Now you're cruising. One of the great strengths of pipelines is how easy they make the process of testing different models.

Until now, you've been using the model step `('clf', OneVsRestClassifier(LogisticRegression()))` in your pipeline.

But what if you want to try a different model? Do you need to build an entirely new pipeline? New nests? New FeatureUnions? Nope! You just have a simple one-line change, as you'll see in this exercise.

In particular, you'll swap out the logistic-regression model and replace it with a [random forest][1] classifier, which uses the statistics of an ensemble of decision trees to generate predictions.

**Instructions**

- Import the `RandomForestClassifier` from `sklearn.ensemble`.
- Add a `RandomForestClassifier()` step named `'clf'` to the pipeline.
- Hit submit to fit the pipeline to the training data and compute its accuracy.


[1]: https://en.wikipedia.org/wiki/Random_forest

In [ ]:
# Edit model step in pipeline
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', RandomForestClassifier(n_jobs=-1))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on budget dataset: ", accuracy)

**An accuracy improvement- amazing! All your work building the pipeline is paying off. It's now very simple to test different models!**

In [ ]:
# Edit model step in pipeline
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', RandomForestClassifier(n_estimators=15, n_jobs=-1))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on budget dataset: ", accuracy)

**It's time to get serious and work with the log loss metric. You'll learn expert techniques in the next chapter to take the model to the next level.**

# Learning from the experts

In this chapter, you will learn the tricks used by the competition winner, and implement them yourself using scikit-learn.

## Learning from the expert: processing

1. Learning from the expert: processing: In this chapter, we're going to see the tips and tricks that got the winning competitor to the top of the leaderboard.

2. Learning from the expert: Some tricks are about text processing, some are about statistical methods, and some are about computational efficiency. While some of this may be new, some of it may be familiar from this course or other work you've done on DataCamp. By knowing which tools to combine, we can build extremely effective models.
- Text processing
- Statistical methods
- Computational efficiency
3. Learning from the expert: text preprocessing: In our introduction to natural language processing, we introduced the same tools that the winner used. These are common tools for manipulating text, and are often effective for improving models based on text data. The first trick the winner used was to tokenize on punctuation. By noticing that there are lots of characters like hyphens, periods, and underscores in the text we are working with, the winner picked out a better way of separating words than just spaces. The other trick was to use a range of n-grams in the model. By including unigrams and bigrams, the winner was more likely to capture important information that appears as multiple tokens in the text--for example, "middle school."
- NLP tricks for text data
  - Tokenize on punctuation to avoid hyphens, underscores, etc.
  - Include unigrams and bi-grams in the model to capture important information involving multiple tokens (e.g., "middle school")
4. N-grams and tokenization: One of the massive benefits of building our models on top of scikit-learn is that many of these common tools are implemented for us and are well-tested by a large community. To change our tokenization and add bi-grams, we can change our call to `CountVectorizer`. We'll pass the regular expression to only accept alphanumeric characters in our tokens. We defined this expression back in chapter 2. We'll also add the parameter `ngram_range` equals (1, 2). This tells the `CountVectorizer` to include 1-grams and 2-grams in the vectorization. These changes to the call to `CountVectorizer` are the only changes we need to make to add these preprocessing steps. We can easily update our pipeline to include this change, which will be part of the exercises.
```python
vec = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC, ngram_range=(1, 2))
```
- Simple changes to `CountVectorizer` can improve your model
  - `token_pattern` parameter to avoid breaking on hyphens
  - `ngram_range` parameter to include unigrams and bigrams
    - `ngram_range=(min_n, max_n)`
  - alphanumeric tokenization
    - `token_pattern=TOKENS_ALPHANUMERIC`
5. Range of n-grams in scikit-learn: Then we fit our updated pipeline in the same way that we've been doing throughout the course: call the fit method on our Pipeline object. Getting predictions from our model is also the same as it has been throughout.

6. Range of n-grams in scikit-learn: We use the predict_proba function to get our class probabilities. Because we have built our preprocessing into our Pipeline, we don't need to do any additional processing on our holdout data when we load it. The pipeline represents the entire prediction process from raw data to class probabilities.
```python
pl.fit(X_train, y_train)

holdout = pd.read_csv('HoldoutData.csv', index_col=0)
predictions = pl.predict_proba(holdout)
prediction_df = pd.DataFrame(columns=pd.get_dummies(
    df[LABELS]).columns, index=holdout.index, data=predictions)
prediction_df.to_csv('predictions.csv')
score = score_submission(pred_path='predictions.csv')
```
7. Let's practice!: Now it's your turn to implement these text processing tricks and see how they improve your score.

In [ ]:
df = pd.read_csv('data/2024-01-19_school_budgeting_with_machine_learning_in_python/TrainingData.csv', index_col=0)
LABELS_td = ['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type', 'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status']
categorize_label = lambda x: x.astype('category')
df[LABELS_td] = df[LABELS_td].apply(categorize_label)
NUMERIC_COLUMNS_td = df.select_dtypes(include='number').columns.tolist()

### How many tokens?

Recall from previous chapters that how you tokenize text affects the n-gram statistics used in your model.

Going forward, you'll use alphanumeric sequences, and only alphanumeric sequences, as tokens. Alphanumeric tokens contain only letters a-z and numbers 0-9 (no other characters). In other words, you'll tokenize on punctuation to generate n-gram statistics.

In this exercise, you'll make sure you remember how to tokenize on punctuation.

Assuming we tokenize on punctuation, accepting only alphanumeric sequences as tokens, how many tokens are in the following string from the main dataset?

`'PLANNING,RES,DEV,& EVAL      '`
If you want, we've loaded this string into the workspace as SAMPLE_STRING, but you may not need it to answer the question.

**Instructions**

- ~~4, because `RES` and `DEV` are not tokens~~
- **4, because `,` and `&` are not tokens** - **Commas, "&", and whitespace are not alphanumeric tokens.**
- ~~7, because there are 4 different words, some commas, an `&` symbol, and whitespace~~
- ~~7, because there are 7 whitespaces~~

### Deciding what's a word

Before you build up to the winning pipeline, it will be useful to look a little deeper into how the text features will be processed.

In this exercise, you will use `CountVectorizer` on the training data `X_train` (preloaded into the workspace) to see the effect of tokenization on punctuation.

Remember, since `CountVectorizer` expects a vector, you'll need to use the preloaded function, `combine_text_columns` before fitting to the training data.

**Instructions**

- Create `text_vector` by preprocessing `X_train` using `combine_text_columns`. This is important, or else you won't get any tokens!
- Instantiate `CountVectorizer` as `text_features`. Specify the keyword argument `token_pattern=TOKENS_ALPHANUMERIC`.
- Fit `text_features` to the `text_vector`.
- Hit submit to print the first 10 tokens.

In [ ]:
# Create the text vector
text_vector = combine_text_columns(X_train)

# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate the CountVectorizer: text_features
text_features = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Fit text_features to the text vector
text_features.fit(text_vector)

# Print the first 10 tokens
print(text_features.get_feature_names_out()[:10])

### N-gram range in scikit-learn

In this exercise, you'll insert a `CountVectorizer` instance into your pipeline for the main dataset, and compute multiple n-gram features to be used in the model.

To look for ngram relationships at multiple scales, you will use the `ngram_range` parameter as Peter discussed in the video.

**Special functions**: You'll notice a couple of new steps provided in the pipeline in this and many of the remaining exercises. Specifically, the `dim_red` step following the `vectorizer` step, and the `scale` step preceding the `clf` (classification) step.

These have been added to account for the fact that you're using a reduced-size sample of the full dataset in this course. To make sure the models perform as the expert competition winner intended, we have to apply a [dimensionality reduction][1] technique, which is what the `dim_red` step does. We have to [scale the features][2] to lie between -1 and 1, which is what the `scale` step does.

The `dim_red` step uses a scikit-learn function called SelectKBest(), applying something called the [chi-squared test][3] to select the K "best" features. The `scale` step uses a scikit-learn function called `MaxAbsScaler()` to squash the relevant features into the interval -1 to 1.

You won't need to do anything extra with these functions here, complete the vectorizing pipeline steps below. However, notice how easy it was to add more processing steps to our pipeline!

**Instructions**

- Import `CountVectorizer` from `sklearn.feature_extraction.text`.
- Add a `CountVectorizer` step to the pipeline with the name `'vectorizer'`.
- Set the token pattern to be `TOKENS_ALPHANUMERIC`.
- Set the `ngram_range` to be `(1, 2)`.


[1]: https://en.wikipedia.org/wiki/Dimensionality_reduction
[2]: https://en.wikipedia.org/wiki/Feature_scaling
[3]: https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
# Select 300 best features
chi_k = 300

# Perform preprocessing
get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS_td], validate=False)

# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate pipeline: pl
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC, ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, k=chi_k))
                ]))
             ]
        )),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# fit the pipeline to our training data
pl.fit(X_train, y_train.values)

# print the score of our trained pipeline on our test set
log_loss_scorer = make_scorer(_multi_multi_log_loss)
print("Logloss score of trained pipeline: ", log_loss_scorer(pl, X_test, y_test.values))

## Learning from the expert: a stats trick

1. **Learning from the expert: a stats trick**: Now that you have experience with the text processing tricks that won the competition, we'll introduce some additional tools that the winner used.

2. **Learning from the expert: interaction terms**: The statistical tool that the winner used is called interaction terms. We use bigrams to account for when words appear in a certain order. However, what if terms are not next to each other? For example, consider "English Teacher for 2nd Grade" and "2nd Grade - budget for English Teacher". If I want to identify this line item as a staff position for an elementary school, it helps to know that both "2nd grade" and "English teacher" appear. Interaction terms let us mathematically describe when tokens appear together.
- Statistical tool that the winner used: interaction terms
- Example
  - English teacher for 2nd grade
  - 2nd grade - budget for English teacher
- Interaction terms mathematically describe when tokens appear together

3. **Interaction terms: the math**: We're going to walk through a brief bit of math here to help explain interaction terms. If we look at a simple linear model, we have

4. **Interaction terms: the math**: Beta-one times X1, beta 2 times X2.

5. **Interaction terms: the math**: Here, X1 and X2 represent whether a particular token appears in the row. The betas are the coefficients, which represent how important that particular token is. To add an interaction term,

6. **Interaction terms: the math**: we add another column to our array, X3 that equals X1 times X2. Because X1 and X2 are either 0 or 1, when we multiply them, we only get a 1 if both occur. This new column, X3, has its own coefficient, Beta 3, which can separately measure how important it is that X1 and X2 appear together. Again,
- ![][sb2]

7. **Adding interaction features with scikit-learn**: scikit-learn provides us with a straightforward way of adding interaction terms. In scikit-learn this functionality is called `PolynomialFeatures`, and we import it from the sklearn dot preprocessing module. As an example, we'll show the same x matrix as the last slide. When we create a `PolynomialFeatures` we tell it the degree of features to include. In our example, we looked at multiplying 2 columns together to see if they co-occurred, so degree equals 2, however, you could 3, 4, or more. While this can sometimes improve the model results, adding larger degrees very quickly scales the number of features outside what is computationally feasible. The `interaction_only` equals True parameter tells `PolynomialFeatures` that we don't need to multiply a column by itself, and we'll talk about include_bias on the next slide. When we fit and transform our array, x, we can see that we get the expected output from the last slide with the new column added. We opted not to include a bias term in our model, but you could have if you wanted to. A bias term is an offset for a model.
```python
from sklearn.preprocessing import PolynomialFeatures

x = pd.DataFrame({'x1': [0, 1], 'x2': [1, 1]})

interaction = PolynomialFeatures(degree=2,
                                 interaction_only=True,
                                 include_bias=False)
interaction.fit_transform(x)

[out]:
array([[0., 1., 0.],
       [1., 1., 1.]])
```
8. **A note about bias terms**: For example, if we plot age on the x-axis, and weights on the y-axis, we'd have a line that had a value 7-point-5 lbs when the baby is born, because a baby has a weight even when it is 0 years old! A bias term allows your model to account for situations where an x value of 0 should have a y value that is not 0, like in this example.
- Bias term allows your model to have non-zero y value when x is 0
- ![][sb3]

9. **Sparse interaction features**: As we mentioned, adding interaction terms makes your X array grow exponentially. Because of computational concerns, our `CountVectorizer` returns an object called a sparse matrix. We won't dig into the details here, but the standard `PolynomialFeatures` object is not compatible with a sparse matrix. We provide you with a replacement, `SparseInteractions` which works with a sparse matrix. You only need to pass it the `degree` parameter for it to work in the same way.
- `SparseInteractions(degree=2).fit_transform(x).toarray()`
- The number of interaction terms grows exponentially
- Our vectorizer saves memory by using a sparse matrix
- `PolynomialFeatures` does not support sparse matrices
- We have provided `SparseInteractions` to work for this problem
10. **Let's practice!**: Now it's your turn to practice adding interaction terms with `SparseInteractions` to your code, and you can see what effect they have on your score!


[sb2]: https://raw.githubusercontent.com/trenton3983/DataCamp/master/Images/2024-01-19_school_budgeting_with_machine_learning_in_python/sb02.jpg
[sb3]: https://raw.githubusercontent.com/trenton3983/DataCamp/master/Images/2024-01-19_school_budgeting_with_machine_learning_in_python/sb03.jpg

### Which models of the data include interaction terms?

Recall from the video that interaction terms involve products of features.

Suppose we have two features `x` and `y`, and we use models that process the features as follows:

1. βx + βy + ββ
2. βxy + βx + βy
3. βx + βy + βx^2 + βy^2

where β is a coefficient in your model (not a feature).

Which expression(s) include interaction terms?

**Answer the question**

- ~~The first expression.~~
- **The second expression.** - **The second expression includes the interaction term `βxy`, where the `xy` term is present, which represents interactions between features**
- ~~The third expression.~~
- ~~The first and third expressions.~~

### Implement interaction modeling in scikit-learn

It's time to add interaction features to your model. The `PolynomialFeatures` object in scikit-learn does just that, but here you're going to use a custom interaction object, `SparseInteractions`. Interaction terms are a statistical tool that lets your model express what happens if two features appear together in the same row.

`SparseInteractions` does the same thing as `PolynomialFeatures`, but it uses sparse matrices to do so. You can get the code for `SparseInteractions` at [this GitHub Gist][1].

`PolynomialFeatures` and `SparseInteractions` both take the argument `degree`, which tells them what polynomial degree of interactions to compute.

You're going to consider interaction terms of `degree=2` in your pipeline. You will insert these steps after the preprocessing steps you've built out so far, but before the classifier steps.

Pipelines with interaction terms take a while to train (since you're making n features into n-squared features!), so as long as you set it up right, we'll do the heavy lifting and tell you what your score is!

**Instructions**

- Add the interaction terms step using `SparseInteractions()` with `degree=2`. Give it a name of `'int'`, and make sure it is after the preprocessing step but before scaling.


[1]: https://github.com/drivendataorg/box-plots-sklearn/blob/master/src/features/SparseInteractions.py

In [ ]:
# Instantiate pipeline: pl
pl = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('imputer', SimpleImputer())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vectorizer', CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                               ngram_range=(1, 2))),
                ('dim_red', SelectKBest(chi2, k=chi_k))
            ]))
        ]
    )),
    ('int', SparseInteractions(degree=2)),
    ('scale', MaxAbsScaler()),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

# fit the pipeline to our training data
pl.fit(X_train, y_train.values)

# print the score of our trained pipeline on our test set
log_loss_scorer = make_scorer(_multi_multi_log_loss)
print("Logloss score of trained pipeline: ", log_loss_scorer(pl, X_test, y_test.values))

## Learning from the expert: the winning model

1. Learning from the expert: the winning model: Congratulations! You've implemented nearly all of the winning pipeline. Just a couple more tweaks.

2. Learning from the expert: hashing trick: We've mentioned that we need to balance adding new features with the computational cost of additional columns. For example, adding 3-grams or 4-grams is going to have an enormous increase in the size of the array. As the array grows in size, we need more computational power to fit our model. The "hashing trick" is a way of limiting the size of the matrix that we create without sacrificing too much model accuracy. A hash function takes an input, in this case a token, and outputs a hash value. For example, the hash value may be an integer like in this example below. We explicitly state how many possible outputs the hashing function may have. For example, we may say that we will only have 250 outputs of the hash function. The `HashingVectorizer` then maps every token to one of those 250 columns. Some columns will have multiple tokens that map to them. Interestingly, the original paper about the hashing function demonstrates that even if two tokens hash to the same value, there is very little effect on model accuracy in real world problems.
- Adding new features may cause enormous increase in array size
- Hashing is a way of increasing memory efficiency
  - ![][sb4]
- Hash function limits possible outputs, fixing array size
3. When to use the hashing trick: We've been working with a small subset of the data for this course, however, the size of the full dataset is part of the challenge. In this situation, we want to do whatever we can to make our array of features smaller. Doing so is called "**dimensionality reduction**". The hashing trick is particularly useful in cases like this where we have a very large amount of text data.
- Want to make array of features as small as possible
  - Dimensionality reduction
  - Particularly useful on large datasets
  - e.g., lots of text data!
4. Implementing the hashing trick in scikit-learn: Implementing the hashing trick is very simple in scikit-learn. Instead of using the `CountVectorizer`, which creates the bag of words representation, we change to the `HashingVectorizer`. We can pass this the same `token_pattern` and `ngram_range` as before. The parameters norm equals None and non_negative equals `True` let you drop in the `HashingVectorizer` as a replacement for the `CountVectorizer`. For more details on those two parameters, you can see the scikit-learn documentation. We can now use the `HashingVectorizer` in a `Pipeline` instead of the `CountVectorizer`.
```python
from sklearn.feature_extraction.text import HashingVectorizer
vec = HashingVectorizer(norm=None, non_negative=True, token_pattern=TOKENS_ALPHANUMERIC, ngram_range=(1, 2))
```
5. The model that won it all: We've worked through the three categories of tools that the winner put together to win the competition. The NLP trick was to use bigrams and tokenize on punctuation. The statistical trick was to add interaction terms to the model. The computational trick was to use a `HashingVectorizer`. You can now code all of these in a scikit-learn Pipeline. However, there's one thing we haven't talked about yet. What is the class of model that the winner used? Was it a deep convolutional neural network? Extreme gradient boosted trees? An ensemble of local-expert elastic net regressions?
- You now know all the expert moves to make on this dataset
  - NLP: Range of n-grams, punctuation tokenization
  - Stats: Interaction terms
  - Computation: Hashing trick
- What class of model was used?
6. The model that won it all: No! The winning model was, in fact, the simple model we started with: **logistic regression**! The competition wasn't won by a cutting edge algorithm. Instead, it was won by thinking carefully about how to create features for the model and then adding a couple of easily implemented tricks. So, instead of reaching for those advanced algorithms that are hard to interpret and expensive to train, it's always worth it to see how far you can get with simpler methods.
- And the winning model was…
- Logistic regression!
  - Carefully create features
  - Easily implemented tricks
  - Favor simplicity over complexity and see how far it takes you!
7. Let's practice!: Now it's your chance to use all these tools together to create the winning Pipeline.

[sb4]: https://raw.githubusercontent.com/trenton3983/DataCamp/master/Images/2024-01-19_school_budgeting_with_machine_learning_in_python/sb04.jpg

### Why is hashing a useful trick?

In the video, Peter explained that a [hash][1] function takes an input, in your case a token, and outputs a hash value. For example, the input may be a string and the hash value may be an integer.

We've loaded a familiar python datatype, a dictionary called `hash_dict`, that makes this mapping concept a bit more explicit. In fact, [python dictionaries ARE hash tables][2]!

Print `hash_dict` in the IPython Shell to get a sense of how strings can be mapped to integers.

By explicitly stating how many possible outputs the hashing function may have, we limit the size of the objects that need to be processed. With these limits known, computation can be made more efficient and we can get results faster, even on large datasets.

Using the above information, answer the following:

Why is hashing a useful trick?

**Instructions**

- ~~Hashing isn't useful unless you're working with numbers.~~
- ~~Some problems are memory-bound and not easily parallelizable, but hashing parallelizes them.~~
- **Some problems are memory-bound and not easily parallelizable, and hashing enforces a fixed length computation instead of using a mutable datatype (like a dictionary)**. - **Enforcing a fixed length can speed up calculations drastically, especially on large datasets!**
- ~~Hashing enforces a mutable length computation instead of using a fixed length datatype, like a dictionary.~~


[1]: https://en.wikipedia.org/wiki/Feature_hashing#Feature_vectorization_using_the_hashing_trick
[2]: http://stackoverflow.com/questions/114830/is-a-python-dictionary-an-example-of-a-hash-table


In [ ]:
hash_dict = {'and': 780, 'fluids': 354, 'fuel': 895, 'petro': 354, 'vend': 785}

### Implementing the hashing trick in scikit-learn

In this exercise you will check out the scikit-learn implementation of `HashingVectorizer` before adding it to your pipeline later.

As you saw in the video, `HashingVectorizer` acts just like `CountVectorizer` in that it can accept `token_pattern` and `ngram_range` parameters. The important difference is that it creates hash values from the text, so that we get all the computational advantages of hashing!

**Instructions**

- Import `HashingVectorizer` from `sklearn.feature_extraction.text`.
- Instantiate the `HashingVectorizer` as hashing_vec using the `TOKENS_ALPHANUMERIC` pattern.
- Fit and transform `hashing_vec` using `text_data`. Save the result as `hashed_text`.
- Hit submit to see some of the resulting hash values.

In [ ]:
# Get text data: text_data
text_data = combine_text_columns(X_train)

# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)' 

# Instantiate the HashingVectorizer: hashing_vec
hashing_vec = HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Fit and transform the Hashing Vectorizer
hashed_text = hashing_vec.fit_transform(text_data)

# Create DataFrame and print the head
hashed_df = pd.DataFrame(hashed_text.data)
print(hashed_df.head())

**As you can see, some text is hashed to the same value, but as Peter mentioned in the video, this doesn't neccessarily hurt performance.**

### Build the winning model

You have arrived! This is where all of your hard work pays off. It's time to build the model that won DrivenData's competition.

You've constructed a robust, powerful pipeline capable of processing training and testing data. Now that you understand the data and know all of the tools you need, you can essentially solve the whole problem in a relatively small number of lines of code. Wow!

All you need to do is add the `HashingVectorizer` step to the pipeline to replace the `CountVectorizer` step.

The parameters `non_negative=True`, `norm=None`, and `binary=False` make the `HashingVectorizer` perform similarly to the default settings on the `CountVectorizer` so you can just replace one with the other.

**Instructions**

- Import `HashingVectorizer` from `sklearn.feature_extraction.text`.
- Add a `HashingVectorizer` step to the pipeline.
  - Name the step `'vectorizer'`.
  - Use the `TOKENS_ALPHANUMERIC` token pattern.
  - Specify the `ngram_range` to be `(1, 2)`

In [ ]:
# Instantiate the winning model pipeline: pl
pl = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('imputer', SimpleImputer())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vectorizer', HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                 alternate_sign=False, norm=None, binary=False,
                                                 ngram_range=(1, 2))),
                ('dim_red', SelectKBest(chi2, k=chi_k))
            ]))
        ]
    )),
    ('int', SparseInteractions(degree=2)),
    ('scale', MaxAbsScaler()),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

# fit the pipeline to our training data
pl.fit(X_train, y_train.values)

# print the score of our trained pipeline on our test set
log_loss_scorer = make_scorer(_multi_multi_log_loss)
print("Logloss score of trained pipeline: ", log_loss_scorer(pl, X_test, y_test.values))

**Log loss: 1.2258. Looks like the performance is about the same, but this is expected since the HashingVectorizer should work the same as the CountVectorizer. Try this pipeline out on the whole dataset on your local machine to see its full power!**

### What tactics got the winner the best score?

Now you've implemented the winning model from start to finish. If you want to use this model locally, [this Jupyter notebook][1] contains all the code you've worked so hard on. You can now take that code and build on it!

Let's take a moment to reflect on why this model did so well. What tactics got the winner the best score?

**Answer the question**

- The winner used a 500 layer deep convolutional neural network to master the budget data.
- The winner used an ensemble of many models for classification, taking the best results as predictions.
- **The winner used skillful NLP, efficient computation, and simple but powerful stats tricks to master the budget data.** - **Often times simpler is better, and understanding the problem in depth leads to simpler solutions!**

[1]: https://github.com/datacamp/course-resources-ml-with-experts-budgets/blob/master/notebooks/1.0-full-model.ipynb

# Next steps and the social impact of your work

1. Next steps and the social impact of your work: Congratulations on finishing this course! I hope you enjoyed it and learned some useful tools to add to your data science toolbox. Your new skills have the ability to really make a difference. The volume of data is growing everywhere, not only in the tech industry. And with this growth in data comes opportunity: the opportunity to use your data science skills to make people's lives better.

2. Can you do better?: One of our goals has been to demonstrate how flexible and powerful the Pipeline API is in scikit-learn. We've successively added small code changes that have had a big effect on our model. Now we want you to think about how you might improve things. Some areas to look at might be: Including stemming or stop-word removal in your natural language processing. Trying a different class of model like RandomForest, KNN or Naive Bayes. Doing further processing on the numeric columns and handling NaNs differently than the default Imputer. One of the benefits of scikit-learn is GridSearch (as you learned about in the supervised learning course). Check out the scikit-learn documentation to see how to perform grid search over every object in a Pipeline. You could even look around the scikit-learn docs and experiment with a new technique you haven't used before. Feel free to sign up on DrivenData and experiment with some of these methods on the full dataset. Learning how these approaches effect real-world data can be a huge boost to your skills.
- You've seen the .exibility of the pipeline steps
- Quickly test ways of improving your submission
  - NLP: Stemming, stop-word removal
  - Model: RandomForest, k-NN, Naïve Bayes
  - Numeric Preprocessing: Imputation strategies
  - Optimization: Grid search over pipeline objects
  - Experiment with new scikit-learn techniques
- Work with the full dataset at DrivenData!
3. Hundreds of hours saved: The goal of building this model was to make an education non-profit more effective in helping school districts with their budgeting decisions. Having a model like the one we have built saves hundreds of hours a year in human time that was spent simply labeling line items. Now budget analysts can spend their time on the decisions that really matter.
- Make schools more e)cient by improving their budgeting decisions
- Saves hundreds of hours each year that humans spent labeling line items
- Can spend more time on the decisions that really ma
4. DrivenData: Data Science to save the world: If you're interested in other ways to use data science to have a social impact, we invite you to check out www-dot-drivendata-dot-org. We've always got competitions just like this running where you can look at new datasets, try interesting methods, learn from the community, and have an impact.
- Other ways to use data science to have a social impact at [DrivenData][1]
- Improve your data science skills while helping meaningful organizations thrive
- Win some cash prizes while you're at it!

[1]: https://www.drivendata.org/

![certificate][1]

[1]: https://raw.githubusercontent.com/trenton3983/DataCamp/master/Images/2024-01-19_school_budgeting_with_machine_learning_in_python/2024-01-27_DataCamp_school_budgeting_with_machine_learning_in_python.jpg